In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
MISSING = -9999

In [3]:
df_train = pd.read_csv("../data/interim/train.csv")
df_test = pd.read_csv("../data/interim/test.csv")

/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
categorical = ['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161', 'cntry']

yes_no_cols = ['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
       'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
       'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
       'v248', 'v254', 'v256', 'v257']

In [5]:
df_train["test"] = 0
df_test["test"] = 1

In [6]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)

In [7]:
df.shape

(39325, 274)

In [8]:
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["2", 2], value=0)
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["1", 1], value=1)

In [9]:
one_hot = categorical + ['v70', 'v71', 'v72', 'v73', 'v102', 'v103', 
                           'v158', 'v159', 'v160', 'v163', 'v164', 
                           'v169', 'v170', 'v190', 'v191',
                          'v216', 'v231']

In [10]:
numeric = [col for col in df.columns if col not in one_hot + ['id', 'satisfied']]

In [11]:
k = df.apply(pd.to_numeric, args=('coerce',)).count().sort_values()

In [21]:
to_impute = set(k[k > 20000].index).intersection(numeric)

In [13]:
for col in one_hot:
    df[col] = pd.Categorical(df[col])
df_encoded = pd.get_dummies(df, columns=one_hot, drop_first=True)

In [14]:
df_encoded = df_encoded.apply(pd.to_numeric, args=('coerce',))

In [15]:
df_encoded[numeric] -= df_encoded[numeric].min()

In [16]:
df_encoded[numeric] /= df_encoded[numeric].max()

In [17]:
df_encoded

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v216_6,v231_.b,v231_.c,v231_.d,v231_1,v231_2,v231_3,v231_4,v231_5,v231_6
0,9948,0.2,0.333333,0.60,0.002325,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,25601,0.4,0.333333,0.44,0.002325,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,8592,0.6,0.333333,0.33,0.002325,0.002325,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,29593,1.0,0.333333,0.08,0.002325,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,4252,0.0,0.000000,0.10,0.002325,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39320,31430,0.0,0.333333,0.13,0.009483,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
39321,3876,0.1,0.666667,0.68,0.009483,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
39322,20710,0.5,0.000000,0.27,0.009483,NaN,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
39323,26015,0.8,0.000000,0.05,0.009483,NaN,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
from sklearn.impute import KNNImputer

In [19]:
imputer = KNNImputer()

In [22]:
df_numeric_imputed = imputer.fit_transform(df_encoded[to_impute])

In [26]:
df_numeric_imputed.shape

(39325, 188)

In [27]:
df_encoded[to_impute].shape

(39325, 188)

In [32]:
df_encoded[list(to_impute)] = df_numeric_imputed

In [35]:
df_encoded = df_encoded.apply(pd.to_numeric, args=('coerce',)).fillna(MISSING)

In [37]:
df_encoded.to_csv("../data/processed/data_encoded_knn_imputed.csv", index=False)

In [38]:
df_encoded

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v216_6,v231_.b,v231_.c,v231_.d,v231_1,v231_2,v231_3,v231_4,v231_5,v231_6
0,9948,0.2,0.333333,0.60,0.002325,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,25601,0.4,0.333333,0.44,0.002325,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,8592,0.6,0.333333,0.33,0.002325,0.002325,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,29593,1.0,0.333333,0.08,0.002325,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,4252,0.0,0.000000,0.10,0.002325,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39320,31430,0.0,0.333333,0.13,0.009483,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
39321,3876,0.1,0.666667,0.68,0.009483,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
39322,20710,0.5,0.000000,0.27,0.009483,-9999.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
39323,26015,0.8,0.000000,0.05,0.009483,-9999.000000,0.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0


In [23]:
# df_numeric = df.apply(pd.to_numeric, args=('coerce',)).fillna(MISSING)

In [24]:
# df_numeric.to_csv("../data/processed/data_encoded.csv", index=False)